In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Washington_State_HDMA-2016.csv')

C:\Users\Wendy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (23,24,25,28,29,30,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Take a look

In [5]:
df.shape

(466566, 47)

In [3]:
df.isna().sum()

tract_to_msamd_income                615
rate_spread                       457928
population                           610
minority_population                  610
number_of_owner_occupied_units       622
number_of_1_to_4_family_units        611
loan_amount_000s                       0
hud_median_family_income             606
applicant_income_000s              62033
state_name                             0
state_abbr                             0
sequence_number                        0
respondent_id                          0
purchaser_type_name                    0
property_type_name                     0
preapproval_name                       0
owner_occupancy_name                   0
msamd_name                         38274
loan_type_name                         0
loan_purpose_name                      0
lien_status_name                       0
hoepa_status_name                      0
edit_status_name                  392061
denial_reason_name_3              465320
denial_reason_na

In [6]:
df['as_of_year'].unique()

array([2016], dtype=int64)

Drop columns that we do not need. DD here: https://cfpb.github.io/api/hmda/fields.html

In [11]:
df1 = df.drop(columns = ['rate_spread','edit_status_name','denial_reason_name_1','denial_reason_name_2',\
                         'denial_reason_name_3','co_applicant_race_name_2','co_applicant_race_name_3',\
                        'co_applicant_race_name_4','co_applicant_race_name_5','applicant_race_name_2',\
                        'applicant_race_name_3','applicant_race_name_4','applicant_race_name_5','msamd_name',\
                         'census_tract_number'])

In [12]:
df1.isna().sum()

tract_to_msamd_income               615
population                          610
minority_population                 610
number_of_owner_occupied_units      622
number_of_1_to_4_family_units       611
loan_amount_000s                      0
hud_median_family_income            606
applicant_income_000s             62033
state_name                            0
state_abbr                            0
sequence_number                       0
respondent_id                         0
purchaser_type_name                   0
property_type_name                    0
preapproval_name                      0
owner_occupancy_name                  0
loan_type_name                        0
loan_purpose_name                     0
lien_status_name                      0
hoepa_status_name                     0
county_name                         367
co_applicant_sex_name                 0
co_applicant_race_name_1              0
co_applicant_ethnicity_name           0
as_of_year                            0


Drop NAs in *country_code*.

In [13]:
df2 = df1.dropna(subset=['county_name'])

In [15]:
df2.shape

(466199, 32)

Fill NAs in *application_income_000s* with the mean value of the county the application is in.

In [16]:
df2['applicant_income_000s'] = df2.groupby(['county_name'])['applicant_income_000s']\
    .transform(lambda x: x.fillna(x.mean()))

C:\Users\Wendy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Drop all other NAs. (0.13% records)

In [19]:
df2 = df2.dropna()

In [21]:
df2.shape

(465940, 32)

In [24]:
(466566 - 465940)/ 466566

0.001341717999168392

In [25]:
df2.isna().sum()

tract_to_msamd_income             0
population                        0
minority_population               0
number_of_owner_occupied_units    0
number_of_1_to_4_family_units     0
loan_amount_000s                  0
hud_median_family_income          0
applicant_income_000s             0
state_name                        0
state_abbr                        0
sequence_number                   0
respondent_id                     0
purchaser_type_name               0
property_type_name                0
preapproval_name                  0
owner_occupancy_name              0
loan_type_name                    0
loan_purpose_name                 0
lien_status_name                  0
hoepa_status_name                 0
county_name                       0
co_applicant_sex_name             0
co_applicant_race_name_1          0
co_applicant_ethnicity_name       0
as_of_year                        0
application_date_indicator        0
applicant_sex_name                0
applicant_race_name_1       

Create Column *loan_id* for later viz in Tableau.

In [26]:
df2['loan_id'] = range(len(df2))

In [27]:
df2.head().transpose()

,0,1,2,3,4
tract_to_msamd_income,121.69,83.37,91.13,146.17,162.47
population,8381,4915,5075,5032,5183
minority_population,23.79,23.99,11.82,8.59,10.5
number_of_owner_occupied_units,2175,1268,1136,1525,1705
number_of_1_to_4_family_units,2660,1777,1838,1820,2104
loan_amount_000s,227,240,241,351,417
hud_median_family_income,73300,57900,73300,73300,78100
applicant_income_000s,116,42,117,315,114
state_name,Washington,Washington,Washington,Washington,Washington
state_abbr,WA,WA,WA,WA,WA


Take only loans with *application_date_indicator* not equal to 2 (meaning "date not available')

In [30]:
df3 = df2[df2['application_date_indicator']!= 2]

In [31]:
df3.shape

(417606, 33)

Export final sets.

In [32]:
df3.to_csv('WA Home Loan Cleaned.csv',index = False)

In [ ]:
df3['']